In [1]:
import json
import requests

url = 'https://www.wikidata.org/w/api.php'
params = {'action':'wbsearchentities', 
          'language':'en',
          'format':'json'}
url_answer = 'https://query.wikidata.org/sparql'
query = 'SELECT ?property ?propertyLabel WHERE { ?property a wikibase:Property . SERVICE wikibase:label { bd:serviceParam wikibase:language "en" .}}'
message = requests.get(url_answer, params={'query': query, 'format': 'json'})
results = message.json()
label_dic = {}
for i in results['results']['bindings']:
    p_value = i['property']['value'][31:]
    label_name = i['propertyLabel']['value']
    label_dic[p_value] = label_name
# list
index_list = []
label_list = []
for keys, value in label_dic.items():
    index_list.append(keys)
    label_list.append(value)

#!pip install spacy-transformers
#!python -m spacy download en_core_web_trf

In [2]:
from sentence_transformers import SentenceTransformer, util
import spacy
import spacy_transformers
nlp = spacy.load("en_core_web_trf")

#Load the model
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-cos-v5')

#Encode labels
label_embeddings = model.encode(label_list)



In [22]:
import time


url = 'https://www.wikidata.org/w/api.php'
params = {'action':'wbsearchentities', 
          'language':'en',
          'format':'json'}
url_answer = 'https://query.wikidata.org/sparql'


def qa(question):
    question_words = ["Is", "Was", "Does", "Can", "Are"]
    question = question
    answer = None
    no_label_list = ['ORDINAL','GPE', 'DATE', 'PRODUCT']
    parse = nlp(question)
    question = question[0].lower() + question[1:]
    entity = ""

    # Use ent parser
    for ent in parse.ents:
        if ent.label_ not in no_label_list:
            entity += ent.text
            break

    # Use capital letters
    if entity == "":
        for count, word in enumerate(parse):
            if word.text[0].isupper() and count >0 and word.text[1].islower():
                if parse[count + 1].text == 'of':
                    entity += word.text + ' ' + 'of' + ' '
                else:
                    entity += word.text + ' '
    try:
        if question[:8] == 'how many':
            query_emb = model.encode(question[8:])
        else:
            query_emb = model.encode(question)
        scores = util.dot_score(query_emb, label_embeddings)[0].cpu().tolist()
        doc_score_pairs = list(zip(label_list, scores))
        doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)[:10]
        params['search'] = entity
        json = requests.get(url,params).json()
        ID1 = json['search']
        for ents in ID1:
            ID1_1 = ents['title']
            if parse[0].text in question_words:
                query = 'SELECT ?answerLabel WHERE { wd:' + ID1_1 + ' ?property ?answer SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }}'
                message = requests.get(url_answer, params={'query': query, 'format': 'json'})
                try:
                    results = message.json()
                except:
                    time.sleep(10)
                    message = requests.get(url_answer, params={'query': query, 'format': 'json'})
                    results = message.json()
                    continue
                if results['results']['bindings'] == []:
                    continue
                else:
                    answer = results['results']['bindings']
                    results['results']['bindings'] = [] 
                    break
            try:
                if results['results']['bindings'] == [] and answer == None:
                    pass
                else: 
                    break
            except UnboundLocalError:
                pass
            ID1_1 = ents['title']
            for i in doc_score_pairs:
                p = index_list[label_list.index(i[0])] 
                query = 'SELECT ?answerLabel WHERE { wd:' + ID1_1 + ' wdt:' + p + ' ?answer SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }}'
                message = requests.get(url_answer, params={'query': query, 'format': 'json'})
                # If request timeout wait 10 seconds to try again.
                try:
                    results = message.json()
                except:
                    time.sleep(10)
                    message = requests.get(url_answer, params={'query': query, 'format': 'json'})
                    results = message.json()
                    continue
                if results['results']['bindings'] == []:
                    continue
                else:
                    answer = results['results']['bindings']
                    results['results']['bindings'] = [] 
                    break
    except:
        pass
    
    if answer == None:
        if parse[0].text in question_words:
            return 'no'
        else: 
            return 'null'
    else:
        itemlist = []   
        if parse[0].text == 'How' and parse[1].text == 'many' and parse[2].text != 'units' and parse[2].text != 'followers' and parse[2].text != 'copies':
            for item in answer:
                itemlist.append(item['answerLabel']['value'])
            return len(itemlist)
        if parse[0].text in question_words:
            for item in answer:
                answer = item['answerLabel']['value']
                if answer in question and answer not in entity:
                    return 'yes'
             
            return 'no'
        else:   
            for item in answer:
                itemlist.append(item['answerLabel']['value'])
            if len(itemlist) == 1:
                return itemlist[0]
            else:
                answer = ''
                for i in itemlist:
                    answer += i + ',' + ' ' 
                return answer[:-2]
        

In [28]:
import json
  
# Opening JSON file
f = open('all-questions-2023.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
counter = 0
print(qa("Is The Sims a published by Riot games"))
print(qa("Is The Sims a published by Maxis"))
print(qa("Is The Sims a dog?"))
for i in data:
    if counter > 50:
        break
    #print(qa(i['string']), i['answer'][0]['string'])
    counter += 1
    
  
# Closing file
f.close()

no
yes
no
